# Cleaning code for Building 59 dataset

The .csv files from the dataset are located on the path declared right below.

In [1]:
# Basic imports
import csv
import numpy as np
import pandas as pd
from pandas import Series
import datetime
import time
import os
from fancyimpute import KNN, MatrixFactorization
import math

path = "../data" #Path with raw csv files


This is the code presented on the paper, we are not able to execute it due to RAM problems, so we will try to transform it

In [8]:


def clean_data_from_path(path):
    files = os.listdir(path)
    path_postprocess = path + "_postprocess"

    #read data files and adjust time format
    for filename in files:
        print(path+'/'+filename)
        df = pd.read_csv(path+'/'+filename)
        df['date'] = pd.to_datetime(df['date']) 
        helper=pd.DataFrame({'date': pd.date_range(df['date'].min(), df['date'].max(), freq='15min')})
        df = pd.merge(df, helper, on='date', how='outer').sort_values('date')
        count_out = Series([0],index=['date']) #count of outlier values
        count_gap = Series([0],index=['date']) #count of gap
        count_outgap = Series([0],index=['date']) #count of large gap (e.g., one day)
        gap_max=Series([0],index=['date']) #maximum gap
        #calculate the count of gap and do the interpolation based on the gap size 
        for i in range(1, len(df.columns)):
            k = 0
            out_gapcount=0
            start_index = {}
            starttime = {}
            end_index = {}
            endtime = {}
            gap = {}
            
    
            if pd.isnull(df.iloc[len(df.index)-1,i]) == True or math.isnan(df.iloc[len(df.index)-1,i])==True:
                df.iloc[len(df.index)-1,i]=0
            for j in range(0, len(df.index)):
                if (pd.isnull(df.iloc[j,i]) or math.isnan(df.iloc[j,i]))and pd.isnull(df.iloc[j-1,i]) == False:
                    starttime[k]=df.iloc[j-1,0] #start time of the gap
                    start_index[k]=j-1
                elif (pd.isnull(df.iloc[j-1,i]) or math.isnan(df.iloc[j-1,i])) and pd.isnull(df.iloc[j,i]) == False:
                    endtime[k]=df.iloc[j,0] #end time of the gap
                    end_index[k]=j
                    k=k+1
            if k != 0:
                for m in range(k):
                    starttime_struct=datetime.datetime.strptime(str(starttime[m]), '%Y-%m-%d %H:%M:%S')
                    endtime_struct = datetime.datetime.strptime(str(endtime[m]), '%Y-%m-%d %H:%M:%S')
                    gap[m]=(endtime_struct-starttime_struct).total_seconds()
                    if  gap[m]<= 3600: #linear interpolation if the gap is less than one hour
                        df.iloc[start_index[m]:end_index[m]+1,i]=df.iloc[start_index[m]:end_index[m]+1,i].interpolate(method='linear')
                    elif gap[m] >3600*24:
                        out_gapcount=out_gapcount+1
                maxgap = max(gap.values())/60
                gap_max=gap_max.append(Series(maxgap,index=[df.columns[i]]))
            outcount=np.sum(df.iloc[:, i]<0)/len(df)
            count_out=count_out.append(Series(outcount, index=[df.columns[i]]))
            count_gap= count_gap.append(Series(k, index=[df.columns[i]]))
            count_outgap = count_outgap.append(Series(out_gapcount,index=[df.columns[i]]))
            df_interpolation=np.array(df.iloc[:,1:])
        df_interpolation= KNN(k=3).fit_transform(df_interpolation) #Apply knn algorithm if the gap is larger than one hour
        unfill_large_gaps(df_filled, df)
        if out_gapcount !=0:
            df_interpolation= MatrixFactorization().fit_transform(df_interpolation) #Apply MF algorithm if the gap is larger than one day         
        df.iloc[:,1:]=df_interpolation
        cols_not_null = (len(df)-df.count(axis=0))/len(df)
        data=pd.DataFrame({'missingrate':cols_not_null,'outrate':count_out,'count_outgap':count_outgap,'count_gap':count_gap,'maxgap':gap_max})
        data.to_csv(path_postprocess+'\\'+'parameter_'+filename, sep=',', header=True, index=True)
        df.to_csv(path_postprocess+'\\'+'data_'+filename, sep=',', header=True, index=False)


We have a problem with ele.csv (energy use), because it doesn't follow the same csv format as the other files: it includes an unnamed column without data. We solve this problem with the following code (run only once)

In [9]:
#datos = pd.read_csv(path+ '/ele.csv')
#datos.drop('Unnamed: 6', axis=1, inplace=True)
#datos = datos.set_index('date')
#datos.to_csv(path+ '/ele.csv')

In [2]:
# The same problem appears in zone_temp_sp_h and zone_temp_sp_c (unnamed features, all of which have more than 50% missing data), we will just drop then
# We also drop the column zone_070_cooling_sp because it has over 97% of missing values in the interval we will consider
datos = pd.read_csv(path+ '/zone_temp_sp_c.csv')
datos = datos.iloc[:,:40]
datos = datos.set_index('date')
#datos.drop('Unnamed: 0', axis=1, inplace=True)
datos.to_csv(path+ '/zone_temp_sp_c.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/zone_temp_sp_c.csv'

In [54]:
datos = pd.read_csv(path+ '/zone_temp_sp_h.csv')
datos = datos.iloc[:,:40]
datos = datos.set_index('date')
#datos.drop('Unnamed: 0', axis=1, inplace=True)

datos.to_csv(path+ '/zone_temp_sp_h.csv')


### Study of null values by column
For a file, we will study the percentage of missing values it includes.

In [2]:
def summary(path, filename):
    print("SUMMARY OF " + filename)
    dataframe = pd.read_csv(path+'/'+filename)
    dataframe=dataframe.set_index('date')
    for i in range(dataframe.shape[1]):
        print(dataframe.columns[i])
        n_miss = dataframe.iloc[:,i].isnull().sum()
        perc = n_miss / dataframe.shape[0] * 100
        print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))

def summary2(dataframe):
    for i in range(dataframe.shape[1]):
        print(dataframe.columns[i])
        n_miss = dataframe.iloc[:,i].isnull().sum()
        perc = n_miss / dataframe.shape[0] * 100
        print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))

In [5]:
summary(path, 'wifi.csv')

SUMMARY OF wifi.csv
wifi_first_south
> 0, Missing: 7344 (7.3%)
wifi_second_south
> 1, Missing: 7344 (7.3%)
wifi_third_south
> 2, Missing: 0 (0.0%)
wifi_fourth_south
> 3, Missing: 0 (0.0%)


## Interpolation depending on the size of the gap:

- If it's smaller than 1h, we use linear interpolation
- If it's bigger than 1 day, we use KNN with n=3


Interpolation with KNN: 

https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/

In [3]:
from sklearn.impute import KNNImputer
files = os.listdir(path)
path_postprocess = path + "_postprocess"
#in the cleaning code we insert the rows that are apparently missing

freqs = {'zone_co2.csv':'1min', 'ele.csv': '15min', 'zone_temp_sp_c.csv':'5min', 'occ.csv':'1min', 'zone_temp_exterior.csv':'1min', 'zone_temp_sp_h.csv':'5min', 'site_weather.csv':'15min', 'wifi.csv': '10min', 'zone_temp_interior.csv':'10min'}

In [4]:
def unfill_large_gaps(df_filled, df):
    
    for i in range(1, len(df.columns)):
        k=0
        start_index = {}
        starttime = {}
        end_index = {}
        endtime = {}
        gap={}
        for j in range(0, len(df.index)):
            if pd.isnull(df.iloc[j,i]) and pd.isnull(df.iloc[j-1,i]) == False:
                starttime[k]=df.iloc[j-1,0]
                start_index[k]=j-1
            elif pd.isnull(df.iloc[j-1,i]) and pd.isnull(df.iloc[j,i]) == False:
                endtime[k]=df.iloc[j,0]
                end_index[k]=j
                k=k+1
        for m in range(k):
            starttime_struct=datetime.datetime.strptime(str(starttime[m]), '%Y-%m-%d %H:%M:%S')
            endtime_struct = datetime.datetime.strptime(str(endtime[m]), '%Y-%m-%d %H:%M:%S')
            gap[m]=(endtime_struct-starttime_struct).total_seconds()
            if  gap[m]>= 3600*24:
                df_filled.iloc[start_index[m]:end_index[m]+1,i-1]= None
                

In [5]:

def get_csv(path, filename, freq):
    print(path+'/'+filename)
    path_postprocess = path+"_postprocess/data_definitivo"+'/'+filename[:-4]+ "_postprocess.csv"
    df = pd.read_csv(path+'/'+filename)
    df['date'] = pd.to_datetime(df['date']) 
    helper=pd.DataFrame({'date': pd.date_range(df['date'].min(), df['date'].max(), freq=freq)})
    df = pd.merge(df, helper, on='date', how='outer').sort_values('date')
    count_out = Series([0],index=['date']) #count of outlier values
    count_gap = Series([0],index=['date']) #count of gap
    count_outgap = Series([0],index=['date']) #count of large gap (e.g., one day)
    gap_max=Series([0],index=['date']) #maximum gap
    out_gapcount=0
    summary2(df)
    #calculate the count of gap and do the interpolation based on the gap size 
    for i in range(1, len(df.columns)):
        print("Estamos en: ", i)
        k = 0
        
        start_index = {}
        starttime = {}
        end_index = {}
        endtime = {}
        gap = {}
        if pd.isnull(df.iloc[len(df.index)-1,i]) == True or math.isnan(df.iloc[len(df.index)-1,i])==True:
            df.iloc[len(df.index)-1,i]=0
        for j in range(0, len(df.index)):
            if (pd.isnull(df.iloc[j,i]) or math.isnan(df.iloc[j,i]))and pd.isnull(df.iloc[j-1,i]) == False:
                starttime[k]=df.iloc[j-1,0] #start time of the gap
                start_index[k]=j-1
            elif (pd.isnull(df.iloc[j-1,i]) or math.isnan(df.iloc[j-1,i])) and pd.isnull(df.iloc[j,i]) == False:
                endtime[k]=df.iloc[j,0] #end time of the gap
                end_index[k]=j
                k=k+1
        if k != 0:
            for m in range(k):
                starttime_struct=datetime.datetime.strptime(str(starttime[m]), '%Y-%m-%d %H:%M:%S')
                endtime_struct = datetime.datetime.strptime(str(endtime[m]), '%Y-%m-%d %H:%M:%S')
                gap[m]=(endtime_struct-starttime_struct).total_seconds()
                if  gap[m]<= 3600: #linear interpolation if the gap is less than one hour
                    print("Interpolation linear")
                    df.iloc[start_index[m]:end_index[m]+1,i]=df.iloc[start_index[m]:end_index[m]+1,i].interpolate(method='linear')
                elif gap[m] >3600*24:
                    out_gapcount=out_gapcount+1
            maxgap = max(gap.values())/60
            gap_max=gap_max.append(Series(maxgap,index=[df.columns[i]]))
        outcount=np.sum(df.iloc[:, i]<0)/len(df)
        count_out=count_out.append(Series(outcount, index=[df.columns[i]]))
        count_gap= count_gap.append(Series(k, index=[df.columns[i]]))
        count_outgap = count_outgap.append(Series(out_gapcount,index=[df.columns[i]]))
    #Interpolate whole dataframe with KNN
    df_interpolated = df.iloc[:,1:]
    imputer = KNNImputer(n_neighbors=3, weights='distance', metric='nan_euclidean')
    imputer.fit(df_interpolated)
    df_interpolated = pd.DataFrame(imputer.transform(df_interpolated), columns=df_interpolated.columns)

    
    #Export into csv
    print("New summary(final): ")
    summary2(df_interpolated)
    df.iloc[:,1:] = df_interpolated
    df.to_csv(path_postprocess, sep=',', header=True, index=False)

    #Final set of information
    cols_not_null = (len(df)-df.count(axis=0))/len(df)
    data=pd.DataFrame({'missingrate':cols_not_null,'outrate':count_out,'count_outgap':count_outgap,'count_gap':count_gap,'maxgap':gap_max})
    return data

However, we have a problem with available data. To solve this problem, we will only keep values between '2018-05-22 07:00:00'
and '2019-02-21 10:11:00' (maximum interval where all the variables are available), in order to do so, we will "crop" the data before interpolating it:


In [33]:

datos = pd.read_csv(path+ '/zone_temp_sp_c.csv')
datos.head(3)

,date,Unnamed: 0,zone_016_cooling_sp,zone_017_cooling_sp,zone_018_cooling_sp,zone_019_cooling_sp,zone_021_cooling_sp,zone_023_cooling_sp,zone_024_cooling_sp,zone_025_cooling_sp,...,zone_059_cooling_sp,zone_061_cooling_sp,zone_062_cooling_sp,zone_063_cooling_sp,zone_064_cooling_sp,zone_065_cooling_sp,zone_066_cooling_sp,zone_067_cooling_sp,zone_069_cooling_sp,zone_070_cooling_sp
0,2018/9/15 10:00,0,73.0,73.0,75.0,78.0,75.0,73.0,73.0,73.0,...,73.0,69.0,69.0,75.0,73.0,73.0,73.0,73.0,75.0,75.0
1,2018/9/15 10:05,1,73.0,73.0,75.0,78.0,75.0,73.0,73.0,73.0,...,73.0,69.0,69.0,75.0,73.0,73.0,73.0,73.0,75.0,75.0
2,2018/9/15 10:10,2,73.0,73.0,75.0,78.0,75.0,73.0,73.0,73.0,...,73.0,69.0,69.0,75.0,73.0,73.0,73.0,73.0,75.0,75.0


In [34]:
datos = pd.read_csv(path+ '/occ.csv')
datos.head(-1) #Decidir si nos quedamos con este límite o pasamos de los datos de ocupación

,date,occ_third_south,occ_fourth_south
0,2018-05-22 07:00:00,0.0,0.0
1,2018-05-22 07:01:00,0.0,1.0
2,2018-05-22 07:02:00,0.0,0.0
3,2018-05-22 07:03:00,0.0,1.0
4,2018-05-22 07:04:00,0.0,0.0
...,...,...,...
396187,2019-02-21 10:07:00,0.0,0.0
396188,2019-02-21 10:08:00,0.0,0.0
396189,2019-02-21 10:09:00,0.0,0.0
396190,2019-02-21 10:10:00,0.0,0.0


In [6]:

def get_csv_cortado(path, filename, freq, start, end):
    print(path+'/'+filename)
    path_postprocess = path+"_postprocess/data_definitivo"+'/'+filename[:-4]+ "_postprocess.csv"
    df = pd.read_csv(path+'/'+filename)
    df['date'] = pd.to_datetime(df['date']) 
    df =df[(df.date>=start)&(df.date<=end)]
    helper=pd.DataFrame({'date': pd.date_range(start, end, freq=freq)})
    df = pd.merge(df, helper, on='date', how='outer').sort_values('date')
    count_out = Series([0],index=['date']) #count of outlier values
    count_gap = Series([0],index=['date']) #count of gap
    count_outgap = Series([0],index=['date']) #count of large gap (e.g., one day)
    gap_max=Series([0],index=['date']) #maximum gap
    out_gapcount=0
    summary2(df)
    #calculate the count of gap and do the interpolation based on the gap size 
    for i in range(1, len(df.columns)):
        print("Estamos en: ", i)
        k = 0
        
        start_index = {}
        starttime = {}
        end_index = {}
        endtime = {}
        gap = {}
        if pd.isnull(df.iloc[len(df.index)-1,i]) == True or math.isnan(df.iloc[len(df.index)-1,i])==True:
            df.iloc[len(df.index)-1,i]=0
        for j in range(0, len(df.index)):
            if (pd.isnull(df.iloc[j,i]) or math.isnan(df.iloc[j,i]))and pd.isnull(df.iloc[j-1,i]) == False:
                starttime[k]=df.iloc[j-1,0] #start time of the gap
                start_index[k]=j-1
            elif (pd.isnull(df.iloc[j-1,i]) or math.isnan(df.iloc[j-1,i])) and pd.isnull(df.iloc[j,i]) == False:
                endtime[k]=df.iloc[j,0] #end time of the gap
                end_index[k]=j
                k=k+1
        if k != 0:
            for m in range(k):
                starttime_struct=datetime.datetime.strptime(str(starttime[m]), '%Y-%m-%d %H:%M:%S')
                endtime_struct = datetime.datetime.strptime(str(endtime[m]), '%Y-%m-%d %H:%M:%S')
                gap[m]=(endtime_struct-starttime_struct).total_seconds()
                if  gap[m]<= 3600: #linear interpolation if the gap is less than one hour
                    print("Interpolation linear")
                    df.iloc[start_index[m]:end_index[m]+1,i]=df.iloc[start_index[m]:end_index[m]+1,i].interpolate(method='linear')
                elif gap[m] >3600*24:
                    out_gapcount=out_gapcount+1
            maxgap = max(gap.values())/60
            gap_max=gap_max.append(Series(maxgap,index=[df.columns[i]]))
        outcount=np.sum(df.iloc[:, i]<0)/len(df)
        count_out=count_out.append(Series(outcount, index=[df.columns[i]]))
        count_gap= count_gap.append(Series(k, index=[df.columns[i]]))
        count_outgap = count_outgap.append(Series(out_gapcount,index=[df.columns[i]]))
    #Interpolate whole dataframe with KNN
    df_interpolated = df.iloc[:,1:]
    imputer = KNNImputer(n_neighbors=3, weights='distance', metric='nan_euclidean')
    imputer.fit(df_interpolated)
    df_interpolated = pd.DataFrame(imputer.transform(df_interpolated), columns=df_interpolated.columns)

    
    #Export into csv
    print("New summary(final): ")
    summary2(df_interpolated)
    df.iloc[:,1:] = df_interpolated
    df.to_csv(path_postprocess, sep=',', header=True, index=False)

    #Final set of information
    cols_not_null = (len(df)-df.count(axis=0))/len(df)
    data=pd.DataFrame({'missingrate':cols_not_null,'outrate':count_out,'count_outgap':count_outgap,'count_gap':count_gap,'maxgap':gap_max})
    return data
    

In [7]:
start = '2018-09-15 10:00:00'
end = '2019-02-21 10:11:00'

In [8]:
files = os.listdir(path)
path_postprocess = path + "_postprocess"

for file in files:
    if file != '.DS_Store':
        print(file, freqs[file])
        get_csv_cortado(path, file, freqs[file], start, end)


zone_temp_exterior.csv 1min
../data/zone_temp_exterior.csv
date
> 0, Missing: 0 (0.0%)
zone_016_temp
> 1, Missing: 43431 (19.0%)
zone_017_temp
> 2, Missing: 43425 (19.0%)
zone_018_temp
> 3, Missing: 43427 (19.0%)
zone_019_temp
> 4, Missing: 45933 (20.1%)
zone_021_temp
> 5, Missing: 43044 (18.8%)
zone_022_temp
> 6, Missing: 44384 (19.4%)
zone_023_temp
> 7, Missing: 44464 (19.4%)
zone_024_temp
> 8, Missing: 44514 (19.4%)
zone_025_temp
> 9, Missing: 45967 (20.1%)
zone_026_temp
> 10, Missing: 43138 (18.8%)
zone_027_temp
> 11, Missing: 46276 (20.2%)
zone_028_temp
> 12, Missing: 45861 (20.0%)
zone_030_temp
> 13, Missing: 43475 (19.0%)
zone_032_temp
> 14, Missing: 46392 (20.3%)
zone_033_temp
> 15, Missing: 45929 (20.1%)
zone_035_temp
> 16, Missing: 43068 (18.8%)
zone_036_temp
> 17, Missing: 43469 (19.0%)
zone_037_temp
> 18, Missing: 43437 (19.0%)
zone_038_temp
> 19, Missing: 43475 (19.0%)
zone_039_temp
> 20, Missing: 43453 (19.0%)
zone_040_temp
> 21, Missing: 45897 (20.0%)
zone_041_temp
> 22,

In [55]:
cooling_a = pd.read_csv(path+'/zone_temp_sp_h.csv')
cooling_a.shape


(237126, 39)

In [56]:
df=cooling_a
df.date=pd.to_datetime(df.date)

In [57]:
df =df[(df.date>=start)&(df.date<=end)]
helper=pd.DataFrame({'date': pd.date_range(start, end, freq='5min')})
df2 = pd.merge(df, helper, on='date', how='outer').sort_values('date')